In [4]:
# =========================================================
# Full Pipeline: Calibration (bias/dark/flat) + Alignment + All-stars differential photometry
# - Inputs:
#     * LIGHT_DIR: raw (unaligned) light FITS
#     * BIAS_DIR / DARK_DIR / FLAT_DIR: optional calibration frames
# - Outputs (under OUTPUT_DIR):
#     * aligned_fits/  : calibrated & aligned FITS (optional)
#     * plots_allstars_lc/: per-star PNG light curves
#     * (optional) CSVs with times/relative flux
# - Notes:
#     * No HOPS dependency; uses astropy, photutils, astroalign, numpy, matplotlib.
#     * Comparison stars: 3 with similar brightness for each target star.
# =========================================================

import os, glob, math, warnings

import numpy as np
import pandas as pd
from astropy.io import fits
from astropy.time import Time
from astropy.stats import sigma_clipped_stats
from photutils.detection import DAOStarFinder
from photutils.aperture import CircularAperture, CircularAnnulus
import matplotlib.pyplot as plt

# Alignment
import astroalign as aa

warnings.filterwarnings("ignore", category=UserWarning)

# ------------------------- User params -------------------------
# --- I/O ---
LIGHT_DIR   = "./Kepler-854b/object"   # 관측(light) 프레임 폴더
BIAS_DIR    = "./Kepler-854b/bias"     # 바이어스 폴더 (없으면 None 또는 빈 폴더)
DARK_DIR    = "./Kepler-854b/dark"    # 다크 폴더
FLAT_DIR    = "./Kepler-854b/flat"    # 플랫 폴더
OUTPUT_DIR  = "./854b_Basic_out1"           # 결과 저장 폴더

# Build what you have
USE_BIAS = True
USE_DARK = True
USE_FLAT = True

# Alignment controls
DO_ALIGNMENT        = True
SAVE_ALIGNED_FITS   = True
ALIGNED_DIR         = os.path.join(OUTPUT_DIR, "aligned_fits")

# Photometry output
PLOT_DIR            = os.path.join(OUTPUT_DIR, "plots_allstars_lc")
SAVE_WIDE_CSV       = True
WIDE_CSV_PATH       = os.path.join(OUTPUT_DIR, "allstars_relflux_wide.csv")
TIME_CSV_PATH       = os.path.join(OUTPUT_DIR, "times_jd.csv")

# Detection/photometry parameters
FWHM_PIX           = 3.5
THRESH_SIGMA       = 5.0
MAX_STARS_DETECT   = 2000
EDGE_MARGIN        = 12
R_AP               = 3.0 * FWHM_PIX
R_IN, R_OUT        = 6.0 * FWHM_PIX, 10.0 * FWHM_PIX
K_COMPS            = 3
BRIGHT_TOL_FRAC    = 0.30
MIN_SEP_PIX        = 3.0 * FWHM_PIX
CLIP_SIGMA         = 4.0

os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(PLOT_DIR, exist_ok=True)
if SAVE_ALIGNED_FITS:
    os.makedirs(ALIGNED_DIR, exist_ok=True)

# ------------------------- Utilities -------------------------
def list_fits_in(dirpath):
    if not os.path.isdir(dirpath):
        return []
    files = sorted(glob.glob(os.path.join(dirpath, "*.fits")) + 
                   glob.glob(os.path.join(dirpath, "*.fit")))
    return files

def read_time_from_header(hdr):
    """Return time as JD. Try keys in priority; fall back to DATE-OBS; else NaN."""
    for key in ["JD", "BJD", "HJD"]:
        if key in hdr:
            try:
                val = float(hdr[key])
                if np.isfinite(val):
                    return val
            except Exception:
                pass
    if "MJD" in hdr:
        try:
            val = float(hdr["MJD"])
            if np.isfinite(val):
                return val + 2400000.5
        except Exception:
            pass
    if "DATE-OBS" in hdr:
        for fmt in ["isot", None]:
            try:
                if fmt == "isot":
                    return Time(hdr["DATE-OBS"], format="isot", scale="utc").jd
                else:
                    return Time(hdr["DATE-OBS"], scale="utc").jd
            except Exception:
                continue
    return np.nan

def load_fits_data(path):
    with fits.open(path) as hdul:
        data = hdul[0].data.astype(float)
        hdr  = hdul[0].header
    return data, hdr

def median_combine(files):
    """Median combine FITS list -> (master, header template or None)."""
    if not files:
        return None, None
    stack = []
    hdr0 = None
    for p in files:
        dat, hdr = load_fits_data(p)
        if hdr0 is None:
            hdr0 = hdr
        stack.append(dat.astype(float))
    master = np.nanmedian(np.stack(stack, axis=0), axis=0)
    return master, hdr0

def build_master_bias(bias_dir):
    files = list_fits_in(bias_dir)
    if not files:
        return None
    mbias, _ = median_combine(files)
    return mbias

def extract_exptime(hdr):
    for key in ["EXPTIME", "EXPOSURE", "EXP_TIME"]:
        if key in hdr:
            try:
                val = float(hdr[key])
                if np.isfinite(val):
                    return val
            except Exception:
                pass
    return None

def build_master_dark_by_exptime(dark_dir):
    """Return dict: exptime -> master_dark (per that exptime)."""
    files = list_fits_in(dark_dir)
    if not files:
        return {}
    # group by exposure time
    by_exp = {}
    for p in files:
        _, hdr = load_fits_data(p)
        expt = extract_exptime(hdr)
        if expt is None:
            continue
        by_exp.setdefault(expt, []).append(p)
    out = {}
    for expt, flist in by_exp.items():
        mdark, _ = median_combine(flist)
        out[expt] = mdark
    return out

def build_master_flat(flat_dir, master_bias=None, dark_dict=None):
    files = list_fits_in(flat_dir)
    if not files:
        return None
    cal_stack = []
    for p in files:
        dat, hdr = load_fits_data(p)
        if master_bias is not None:
            dat = dat - master_bias
        if dark_dict is not None and len(dark_dict) > 0:
            expt = extract_exptime(hdr)
            if expt is not None:
                # pick nearest dark exposure, scale by ratio
                nearest = min(dark_dict.keys(), key=lambda k: abs(k - expt))
                scale = expt / nearest if nearest and nearest != 0 else 1.0
                dat = dat - dark_dict[nearest] * scale
        cal_stack.append(dat)
    mflat = np.nanmedian(np.stack(cal_stack, axis=0), axis=0)
    # normalize
    med = np.nanmedian(mflat[np.isfinite(mflat)])
    if med and np.isfinite(med) and med != 0:
        mflat = mflat / med
    return mflat

def calibrate_frame(data, hdr, master_bias=None, dark_dict=None, flat_norm=None):
    """(raw - bias - scaled dark) / flat_norm"""
    out = data.astype(float).copy()
    if master_bias is not None:
        out = out - master_bias
    if dark_dict is not None and len(dark_dict) > 0:
        expt = extract_exptime(hdr)
        if expt is not None:
            nearest = min(dark_dict.keys(), key=lambda k: abs(k - expt))
            scale = expt / nearest if nearest and nearest != 0 else 1.0
            out = out - dark_dict[nearest] * scale
    if flat_norm is not None:
        with np.errstate(divide="ignore", invalid="ignore"):
            out = out / flat_norm
    return out

def align_to_reference(src_img, ref_img):
    """Return (aligned_image, transform)."""
    try:
        aligned, tf = aa.register(src_img, ref_img, detection_sigma=3.0, max_control_points=50)
        return aligned.astype(float), tf
    except aa.MaxIterError as e:
        raise RuntimeError(f"Alignment failed (MaxIterError): {e}")
    except Exception as e:
        raise RuntimeError(f"Alignment failed: {e}")

def detect_stars(ref_img):
    mean, med, std = sigma_clipped_stats(ref_img, sigma=3.0, maxiters=5)
    dao = DAOStarFinder(fwhm=FWHM_PIX, threshold=THRESH_SIGMA * std)
    tbl = dao(ref_img - med)
    if tbl is None or len(tbl) == 0:
        raise RuntimeError("No stars detected. Adjust FWHM/THRESH_SIGMA/FWHM_PIX.")
    tbl.sort("flux")
    tbl = tbl[::-1]
    if len(tbl) > MAX_STARS_DETECT:
        tbl = tbl[:MAX_STARS_DETECT]
    xyf = np.vstack([tbl["xcentroid"].data, tbl["ycentroid"].data, tbl["flux"].data]).T
    H, W = ref_img.shape
    m = (xyf[:,0] > EDGE_MARGIN) & (xyf[:,0] < W-EDGE_MARGIN) & (xyf[:,1] > EDGE_MARGIN) & (xyf[:,1] < H-EDGE_MARGIN)
    return xyf[m]

def measure_frame_photometry(img, xy):
    apert = CircularAperture(xy, r=R_AP)
    ann   = CircularAnnulus(xy, r_in=R_IN, r_out=R_OUT)
    ap_masks  = apert.to_mask(method="exact")
    ann_masks = ann.to_mask(method="exact")

    sky_vals = []
    for m in ann_masks:
        ann_data = m.multiply(img)
        mask = (ann_data == 0) | ~np.isfinite(ann_data)
        sky_vals.append(np.nanmedian(ann_data[~mask]) if np.any(~mask) else 0.0)
    sky_vals = np.array(sky_vals, dtype=float)

    fluxes = []
    for (m, sky) in zip(ap_masks, sky_vals):
        ap_data = m.multiply(img)
        mask = (ap_data == 0) | ~np.isfinite(ap_data)
        pix = ap_data[~mask]
        area = np.sum(~mask)
        if area == 0:
            fluxes.append(np.nan)
        else:
            fluxes.append(np.nansum(pix) - sky * area)
    return np.array(fluxes, dtype=float)

def pick_comps_for_target(target_idx, med_flux, xy, k=K_COMPS):
    tflux = med_flux[target_idx]
    tx, ty = xy[target_idx, 0], xy[target_idx, 1]
    lower, upper = (1.0 - BRIGHT_TOL_FRAC) * tflux, (1.0 + BRIGHT_TOL_FRAC) * tflux
    cand = []
    for j in range(len(med_flux)):
        if j == target_idx:
            continue
        if not np.isfinite(med_flux[j]):
            continue
        if (med_flux[j] >= lower) and (med_flux[j] <= upper):
            dx = xy[j,0] - tx
            dy = xy[j,1] - ty
            if math.hypot(dx, dy) >= MIN_SEP_PIX:
                cand.append((j, abs(med_flux[j] - tflux)))
    cand.sort(key=lambda t: t[1])
    return [c[0] for c in cand[:k]]

def robust_rel_flux(target_series, comps_series):
    denom = np.nansum(comps_series, axis=1)
    rel = target_series / denom
    med = np.nanmedian(rel)
    reln = rel / med if np.isfinite(med) and med != 0 else rel
    mu, sig = np.nanmedian(reln), np.nanstd(reln)
    ok = np.abs(reln - mu) < CLIP_SIGMA * sig if np.isfinite(sig) and sig > 0 else np.isfinite(reln)
    return reln, ok

def plot_lightcurve(times, rel_flux, ok_mask, title, outpath, comps_ids=None):
    t0 = np.nanmin(times)
    xh = (times - t0) * 24.0
    plt.figure(figsize=(7.2, 4.2))
    plt.scatter(xh[~ok_mask], rel_flux[~ok_mask], s=14, marker='x', alpha=0.6, label="clipped")
    plt.plot(xh[ok_mask], rel_flux[ok_mask], 'o', ms=3, label="data")
    plt.xlabel("Time since first frame [hr]")
    plt.ylabel("Relative flux (ensemble norm.)")
    if comps_ids is not None:
        sub = f" / comps: {','.join(map(str, comps_ids))}"
    else:
        sub = ""
    plt.title(f"{title}{sub}")
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(outpath, dpi=180)
    plt.close()

# ------------------------- Pipeline -------------------------
def main():
    # 0) Gather raw lights
    light_files = list_fits_in(LIGHT_DIR)
    if not light_files:
        raise FileNotFoundError(f"No light frames in {LIGHT_DIR}")
    print(f"Found {len(light_files)} light frames.")

    # 1) Build masters
    mbias = build_master_bias(BIAS_DIR) if USE_BIAS else None
    dark_dict = build_master_dark_by_exptime(DARK_DIR) if USE_DARK else {}
    mflat = build_master_flat(FLAT_DIR, master_bias=mbias, dark_dict=dark_dict) if USE_FLAT else None

    def _brief(name, arr):
        if arr is None or (isinstance(arr, dict) and len(arr)==0):
            return f"{name}: None"
        if isinstance(arr, dict):
            keys = sorted(list(arr.keys()))
            return f"{name}: {len(keys)} exptimes -> {keys[:5]}{'...' if len(keys)>5 else ''}"
        return f"{name}: shape={arr.shape}, med={np.nanmedian(arr):.3f}, std={np.nanstd(arr):.3f}"

    print(_brief("Master Bias", mbias))
    print(_brief("Master Dark dict", dark_dict))
    print(_brief("Master Flat (norm)", mflat))

    # 2) Calibrate & align (streaming)
    ref_data_raw, ref_hdr = load_fits_data(light_files[0])
    ref_cal = calibrate_frame(ref_data_raw, ref_hdr, master_bias=mbias, dark_dict=dark_dict, flat_norm=mflat)

    if DO_ALIGNMENT:
        ref_img = ref_cal
    else:
        ref_img = ref_cal  # still use calibrated first frame as reference

    if SAVE_ALIGNED_FITS:
        h = ref_hdr.copy()
        h["HISTORY"] = "calibrated; reference frame; aligned identity"
        fits.writeto(os.path.join(ALIGNED_DIR, os.path.basename(light_files[0])), ref_img.astype(np.float32), h, overwrite=True)

    # 3) Detect stars on reference
    xyf = detect_stars(ref_img)
    xy  = xyf[:, :2]
    print(f"Detected {len(xy)} stars on reference.")

    # 4) For each frame: calibrate -> align -> photometry
    times = []
    rows = []  # list of flux arrays length M
    skipped = 0

    for idx, p in enumerate(light_files):
        data, hdr = load_fits_data(p)
        cal = calibrate_frame(data, hdr, master_bias=mbias, dark_dict=dark_dict, flat_norm=mflat)

        if DO_ALIGNMENT and idx > 0:
            try:
                aligned, tf = align_to_reference(cal, ref_img)
            except Exception as e:
                print(f"[WARN] Align failed at {os.path.basename(p)}: {e}")
                skipped += 1
                continue
        else:
            aligned = cal

        # time
        t_jd = read_time_from_header(hdr)
        if not np.isfinite(t_jd):
            t_jd = np.nan  # will fix later with index fallback
        times.append(t_jd)

        # photometry
        fluxes = measure_frame_photometry(aligned, xy)  # length M
        rows.append(fluxes)

        # save aligned fits
        if SAVE_ALIGNED_FITS:
            h = hdr.copy()
            h["HISTORY"] = "calibrated & aligned"
            fits.writeto(os.path.join(ALIGNED_DIR, os.path.basename(p)), aligned.astype(np.float32), h, overwrite=True)

        if (idx+1) % 10 == 0:
            print(f" processed {idx+1}/{len(light_files)} frames...")

    if skipped > 0:
        print(f"Alignment skipped {skipped} frames due to errors.")

    # 5) Assemble flux matrix and times
    flux_mat = np.vstack(rows)   # (N_valid, M)
    times = np.array(times, dtype=float)
    if np.any(~np.isfinite(times)):
        # replace NaN times with indices
        times = np.arange(len(times), dtype=float)

    # quality filter on stars
    valid_ratio = np.mean(np.isfinite(flux_mat), axis=0)
    keep = valid_ratio > 0.5
    xy, flux_mat = xy[keep], flux_mat[:, keep]
    print(f"Kept {xy.shape[0]} stars after quality mask.")

    # 6) Build relative flux per star using 3 comps each + PNGs
    med_flux = np.nanmedian(flux_mat, axis=0)
    saved = 0
    rel_wide = {}  # star_i -> rel_flux array

    for ti in range(xy.shape[0]):
        comp_ids = pick_comps_for_target(ti, med_flux, xy, k=K_COMPS)
        if len(comp_ids) < K_COMPS:
            continue
        target_series = flux_mat[:, ti]
        comps_series  = flux_mat[:, comp_ids]    # (N, K)
        rel, ok = robust_rel_flux(target_series, comps_series)

        # save PNG
        outpng = os.path.join(PLOT_DIR, f"lc_star{ti:04d}.png")
        title  = f"Star {ti:04d} @ (x={xy[ti,0]:.1f}, y={xy[ti,1]:.1f})"
        plot_lightcurve(times, rel, ok, title, outpng, comps_ids=[int(i) for i in comp_ids])
        rel_wide[f"star{ti:04d}"] = rel
        saved += 1

    print(f"Saved {saved} light-curve PNGs to: {PLOT_DIR}")

    # 7) Optional CSVs
    if SAVE_WIDE_CSV and saved > 0:
        # times
        pd.DataFrame({"JD": times}).to_csv(TIME_CSV_PATH, index=False)
        # rel flux wide
        wide = pd.DataFrame(rel_wide)
        wide.insert(0, "JD", times)
        wide.to_csv(WIDE_CSV_PATH, index=False)
        print(f"Wrote CSVs: {TIME_CSV_PATH} , {WIDE_CSV_PATH}")

if __name__ == "__main__":
    main()


Found 95 light frames.
Master Bias: shape=(2048, 2048), med=1100.000, std=7.919
Master Dark dict: 1 exptimes -> [180.0]
Master Flat (norm): shape=(2048, 2048), med=1.000, std=0.007
Detected 197 stars on reference.
 processed 10/95 frames...
 processed 20/95 frames...
 processed 30/95 frames...
 processed 40/95 frames...
 processed 50/95 frames...
 processed 60/95 frames...
 processed 70/95 frames...
 processed 80/95 frames...
 processed 90/95 frames...
Kept 197 stars after quality mask.
Saved 189 light-curve PNGs to: ./854b_Basic_out1\plots_allstars_lc
Wrote CSVs: ./854b_Basic_out1\times_jd.csv , ./854b_Basic_out1\allstars_relflux_wide.csv


In [5]:
# =========================================================
# Full Pipeline (Calib + Align + Photometry) with "detected stars" annotated preview
# =========================================================

import os, glob, math, warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from astropy.io import fits
from astropy.time import Time
from astropy.stats import sigma_clipped_stats

from photutils.detection import DAOStarFinder
from photutils.aperture import CircularAperture, CircularAnnulus

import astroalign as aa
warnings.filterwarnings("ignore", category=UserWarning)

# ------------------------- User params -------------------------
LIGHT_DIR   = "./Kepler-854b/object"   # 관측(light) 프레임 폴더
BIAS_DIR    = "./Kepler-854b/bias"     # 바이어스 폴더 (없으면 None 또는 빈 폴더)
DARK_DIR    = "./Kepler-854b/dark"    # 다크 폴더
FLAT_DIR    = "./Kepler-854b/flat"    # 플랫 폴더
OUTPUT_DIR  = "./854b_Basic_out2"           # 결과 저장 폴더

USE_BIAS = True
USE_DARK = True
USE_FLAT = True

DO_ALIGNMENT        = True
SAVE_ALIGNED_FITS   = True
ALIGNED_DIR         = os.path.join(OUTPUT_DIR, "aligned_fits")

PLOT_DIR            = os.path.join(OUTPUT_DIR, "plots_allstars_lc")
SAVE_WIDE_CSV       = True
WIDE_CSV_PATH       = os.path.join(OUTPUT_DIR, "allstars_relflux_wide.csv")
TIME_CSV_PATH       = os.path.join(OUTPUT_DIR, "times_jd.csv")

# Detection/photometry parameters
FWHM_PIX           = 3.5
THRESH_SIGMA       = 5.0
MAX_STARS_DETECT   = 2000
EDGE_MARGIN        = 12
R_AP               = 3.0 * FWHM_PIX
R_IN, R_OUT        = 6.0 * FWHM_PIX, 10.0 * FWHM_PIX
K_COMPS            = 3
BRIGHT_TOL_FRAC    = 0.30
MIN_SEP_PIX        = 3.0 * FWHM_PIX
CLIP_SIGMA         = 4.0

# Preview rendering
N_LABELS_PREVIEW   = 100   # annotate first N stars (brightest first)
PREVIEW_PATH       = os.path.join(OUTPUT_DIR, "detected_stars_preview.png")

os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(PLOT_DIR, exist_ok=True)
if SAVE_ALIGNED_FITS:
    os.makedirs(ALIGNED_DIR, exist_ok=True)

# ------------------------- Utilities -------------------------
def list_fits_in(dirpath):
    if not os.path.isdir(dirpath):
        return []
    files = sorted(glob.glob(os.path.join(dirpath, "*.fits")) + 
                   glob.glob(os.path.join(dirpath, "*.fit")))
    return files

def read_time_from_header(hdr):
    for key in ["JD", "BJD", "HJD"]:
        if key in hdr:
            try:
                val = float(hdr[key])
                if np.isfinite(val):
                    return val
            except Exception:
                pass
    if "MJD" in hdr:
        try:
            val = float(hdr["MJD"])
            if np.isfinite(val):
                return val + 2400000.5
        except Exception:
            pass
    if "DATE-OBS" in hdr:
        for fmt in ["isot", None]:
            try:
                if fmt == "isot":
                    return Time(hdr["DATE-OBS"], format="isot", scale="utc").jd
                else:
                    return Time(hdr["DATE-OBS"], scale="utc").jd
            except Exception:
                continue
    return np.nan

def load_fits_data(path):
    with fits.open(path) as hdul:
        data = hdul[0].data.astype(float)
        hdr  = hdul[0].header
    return data, hdr

def median_combine(files):
    if not files:
        return None, None
    stack = []
    hdr0 = None
    for p in files:
        dat, hdr = load_fits_data(p)
        if hdr0 is None:
            hdr0 = hdr
        stack.append(dat.astype(float))
    master = np.nanmedian(np.stack(stack, axis=0), axis=0)
    return master, hdr0

def build_master_bias(bias_dir):
    files = list_fits_in(bias_dir)
    if not files:
        return None
    mbias, _ = median_combine(files)
    return mbias

def extract_exptime(hdr):
    for key in ["EXPTIME", "EXPOSURE", "EXP_TIME"]:
        if key in hdr:
            try:
                val = float(hdr[key])
                if np.isfinite(val):
                    return val
            except Exception:
                pass
    return None

def build_master_dark_by_exptime(dark_dir):
    files = list_fits_in(dark_dir)
    if not files:
        return {}
    by_exp = {}
    for p in files:
        _, hdr = load_fits_data(p)
        expt = extract_exptime(hdr)
        if expt is None:
            continue
        by_exp.setdefault(expt, []).append(p)
    out = {}
    for expt, flist in by_exp.items():
        mdark, _ = median_combine(flist)
        out[expt] = mdark
    return out

def build_master_flat(flat_dir, master_bias=None, dark_dict=None):
    files = list_fits_in(flat_dir)
    if not files:
        return None
    cal_stack = []
    for p in files:
        dat, hdr = load_fits_data(p)
        if master_bias is not None:
            dat = dat - master_bias
        if dark_dict is not None and len(dark_dict) > 0:
            expt = extract_exptime(hdr)
            if expt is not None:
                nearest = min(dark_dict.keys(), key=lambda k: abs(k - expt))
                scale = expt / nearest if nearest and nearest != 0 else 1.0
                dat = dat - dark_dict[nearest] * scale
        cal_stack.append(dat)
    mflat = np.nanmedian(np.stack(cal_stack, axis=0), axis=0)
    med = np.nanmedian(mflat[np.isfinite(mflat)])
    if med and np.isfinite(med) and med != 0:
        mflat = mflat / med
    return mflat

def calibrate_frame(data, hdr, master_bias=None, dark_dict=None, flat_norm=None):
    out = data.astype(float).copy()
    if master_bias is not None:
        out = out - master_bias
    if dark_dict is not None and len(dark_dict) > 0:
        expt = extract_exptime(hdr)
        if expt is not None:
            nearest = min(dark_dict.keys(), key=lambda k: abs(k - expt))
            scale = expt / nearest if nearest and nearest != 0 else 1.0
            out = out - dark_dict[nearest] * scale
    if flat_norm is not None:
        with np.errstate(divide="ignore", invalid="ignore"):
            out = out / flat_norm
    return out

def align_to_reference(src_img, ref_img):
    try:
        aligned, tf = aa.register(src_img, ref_img, detection_sigma=3.0, max_control_points=50)
        return aligned.astype(float), tf
    except aa.MaxIterError as e:
        raise RuntimeError(f"Alignment failed (MaxIterError): {e}")
    except Exception as e:
        raise RuntimeError(f"Alignment failed: {e}")

def detect_stars(ref_img):
    mean, med, std = sigma_clipped_stats(ref_img, sigma=3.0, maxiters=5)
    dao = DAOStarFinder(fwhm=FWHM_PIX, threshold=THRESH_SIGMA * std)
    tbl = dao(ref_img - med)
    if tbl is None or len(tbl) == 0:
        raise RuntimeError("No stars detected. Adjust FWHM/THRESH_SIGMA/FWHM_PIX.")
    tbl.sort("flux")
    tbl = tbl[::-1]
    if len(tbl) > MAX_STARS_DETECT:
        tbl = tbl[:MAX_STARS_DETECT]
    xyf = np.vstack([tbl["xcentroid"].data, tbl["ycentroid"].data, tbl["flux"].data]).T
    H, W = ref_img.shape
    m = (xyf[:,0] > EDGE_MARGIN) & (xyf[:,0] < W-EDGE_MARGIN) & (xyf[:,1] > EDGE_MARGIN) & (xyf[:,1] < H-EDGE_MARGIN)
    return xyf[m]

def measure_frame_photometry(img, xy):
    apert = CircularAperture(xy, r=R_AP)
    ann   = CircularAnnulus(xy, r_in=R_IN, r_out=R_OUT)
    ap_masks  = apert.to_mask(method="exact")
    ann_masks = ann.to_mask(method="exact")

    sky_vals = []
    for m in ann_masks:
        ann_data = m.multiply(img)
        mask = (ann_data == 0) | ~np.isfinite(ann_data)
        sky_vals.append(np.nanmedian(ann_data[~mask]) if np.any(~mask) else 0.0)
    sky_vals = np.array(sky_vals, dtype=float)

    fluxes = []
    for (m, sky) in zip(ap_masks, sky_vals):
        ap_data = m.multiply(img)
        mask = (ap_data == 0) | ~np.isfinite(ap_data)
        pix = ap_data[~mask]
        area = np.sum(~mask)
        if area == 0:
            fluxes.append(np.nan)
        else:
            fluxes.append(np.nansum(pix) - sky * area)
    return np.array(fluxes, dtype=float)

def pick_comps_for_target(target_idx, med_flux, xy, k=K_COMPS):
    tflux = med_flux[target_idx]
    tx, ty = xy[target_idx, 0], xy[target_idx, 1]
    lower, upper = (1.0 - BRIGHT_TOL_FRAC) * tflux, (1.0 + BRIGHT_TOL_FRAC) * tflux
    cand = []
    for j in range(len(med_flux)):
        if j == target_idx:
            continue
        if not np.isfinite(med_flux[j]):
            continue
        if (med_flux[j] >= lower) and (med_flux[j] <= upper):
            dx = xy[j,0] - tx
            dy = xy[j,1] - ty
            if math.hypot(dx, dy) >= MIN_SEP_PIX:
                cand.append((j, abs(med_flux[j] - tflux)))
    cand.sort(key=lambda t: t[1])
    return [c[0] for c in cand[:k]]

def robust_rel_flux(target_series, comps_series):
    denom = np.nansum(comps_series, axis=1)
    rel = target_series / denom
    med = np.nanmedian(rel)
    reln = rel / med if np.isfinite(med) and med != 0 else rel
    mu, sig = np.nanmedian(reln), np.nanstd(reln)
    ok = np.abs(reln - mu) < CLIP_SIGMA * sig if np.isfinite(sig) and sig > 0 else np.isfinite(reln)
    return reln, ok

def _stretch(img, p_lo=1, p_hi=99):
    """Percentile stretch for nicer preview."""
    finite = np.isfinite(img)
    if not np.any(finite):
        return img
    v1, v2 = np.percentile(img[finite], [p_lo, p_hi])
    v1, v2 = float(v1), float(v2)
    out = np.clip((img - v1) / max(v2 - v1, 1e-9), 0, 1)
    return out

def save_detection_preview(ref_img, xy, path=PREVIEW_PATH, n_labels=N_LABELS_PREVIEW):
    """Save a single image: background = ref_img, overlay = aperture/annulus + star indices."""
    disp = _stretch(ref_img)
    fig, ax = plt.subplots(figsize=(8, 6))
    im = ax.imshow(disp, cmap="gray", origin="lower")
    cbar = plt.colorbar(im, ax=ax)
    cbar.set_label("stretched intensity")

    # draw rings for first n_labels (or all, if smaller)
    n = min(n_labels, xy.shape[0])
    # Aperture & annulus objects for plotting
    apert = CircularAperture(xy[:n], r=R_AP)
    ann   = CircularAnnulus(xy[:n], r_in=R_IN, r_out=R_OUT)
    try:
        apert.plot(ax=ax, lw=1.2, color="cyan")
        ann.plot(ax=ax, lw=1.0, color="lime")
    except Exception:
        # fallback: just draw apertures if annulus plotting fails
        apert.plot(ax=ax, lw=1.2, color="cyan")

    # labels
    for i in range(n):
        x, y = xy[i]
        ax.text(x+5, y+5, f"{i}", color="yellow", fontsize=9, weight="bold", ha="left", va="bottom")

    ax.set_title(f"Detected stars (N={xy.shape[0]}), aperture/annulus rings")
    ax.set_xlim(0, ref_img.shape[1])
    ax.set_ylim(0, ref_img.shape[0])
    plt.tight_layout()
    plt.savefig(path, dpi=160)
    plt.close()
    return path

def plot_lightcurve(times, rel_flux, ok_mask, title, outpath, comps_ids=None):
    t0 = np.nanmin(times)
    xh = (times - t0) * 24.0
    plt.figure(figsize=(7.2, 4.2))
    plt.scatter(xh[~ok_mask], rel_flux[~ok_mask], s=14, marker='x', alpha=0.6, label="clipped")
    plt.plot(xh[ok_mask], rel_flux[ok_mask], 'o', ms=3, label="data")
    plt.xlabel("Time since first frame [hr]")
    plt.ylabel("Relative flux (ensemble norm.)")
    if comps_ids is not None:
        sub = f" / comps: {','.join(map(str, comps_ids))}"
    else:
        sub = ""
    plt.title(f"{title}{sub}")
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(outpath, dpi=180)
    plt.close()

# ------------------------- Pipeline -------------------------
def main():
    light_files = list_fits_in(LIGHT_DIR)
    if not light_files:
        raise FileNotFoundError(f"No light frames in {LIGHT_DIR}")
    print(f"Found {len(light_files)} light frames.")

    # 1) Masters
    mbias = build_master_bias(BIAS_DIR) if USE_BIAS else None
    dark_dict = build_master_dark_by_exptime(DARK_DIR) if USE_DARK else {}
    mflat = build_master_flat(FLAT_DIR, master_bias=mbias, dark_dict=dark_dict) if USE_FLAT else None

    # 2) Calibrate first as reference
    ref_raw, ref_hdr = load_fits_data(light_files[0])
    ref_cal = calibrate_frame(ref_raw, ref_hdr, master_bias=mbias, dark_dict=dark_dict, flat_norm=mflat)
    ref_img = ref_cal

    if SAVE_ALIGNED_FITS:
        h = ref_hdr.copy()
        h["HISTORY"] = "calibrated; reference frame; aligned identity"
        fits.writeto(os.path.join(ALIGNED_DIR, os.path.basename(light_files[0])),
                     ref_img.astype(np.float32), h, overwrite=True)

    # 3) Detect stars & save annotated preview
    xyf = detect_stars(ref_img)
    xy  = xyf[:, :2]
    print(f"Detected {len(xy)} stars on reference.")
    prev_path = save_detection_preview(ref_img, xy, PREVIEW_PATH, N_LABELS_PREVIEW)
    print(f"Preview saved: {prev_path}")

    # 4) Iterate frames: calibrate, align (to ref), photometry
    times = []
    rows = []
    skipped = 0

    for idx, p in enumerate(light_files):
        data, hdr = load_fits_data(p)
        cal = calibrate_frame(data, hdr, master_bias=mbias, dark_dict=dark_dict, flat_norm=mflat)

        if DO_ALIGNMENT and idx > 0:
            try:
                aligned, tf = align_to_reference(cal, ref_img)
            except Exception as e:
                print(f"[WARN] Align failed at {os.path.basename(p)}: {e}")
                skipped += 1
                continue
        else:
            aligned = cal

        t_jd = read_time_from_header(hdr)
        if not np.isfinite(t_jd):
            t_jd = np.nan
        times.append(t_jd)

        fluxes = measure_frame_photometry(aligned, xy)  # length M
        rows.append(fluxes)

        if SAVE_ALIGNED_FITS:
            h = hdr.copy()
            h["HISTORY"] = "calibrated & aligned"
            fits.writeto(os.path.join(ALIGNED_DIR, os.path.basename(p)),
                         aligned.astype(np.float32), h, overwrite=True)

        if (idx+1) % 10 == 0:
            print(f" processed {idx+1}/{len(light_files)} frames...")

    if skipped > 0:
        print(f"Alignment skipped {skipped} frames due to errors.")

    flux_mat = np.vstack(rows)
    times = np.array(times, dtype=float)
    if np.any(~np.isfinite(times)):
        times = np.arange(len(times), dtype=float)

    # Filter stars
    valid_ratio = np.mean(np.isfinite(flux_mat), axis=0)
    keep = valid_ratio > 0.5
    xy, flux_mat = xy[keep], flux_mat[:, keep]
    print(f"Kept {xy.shape[0]} stars after quality mask.")

    # Build relative flux and PNGs
    med_flux = np.nanmedian(flux_mat, axis=0)
    saved = 0
    rel_wide = {}

    for ti in range(xy.shape[0]):
        comp_ids = pick_comps_for_target(ti, med_flux, xy, k=K_COMPS)
        if len(comp_ids) < K_COMPS:
            continue
        target_series = flux_mat[:, ti]
        comps_series  = flux_mat[:, comp_ids]
        rel, ok = robust_rel_flux(target_series, comps_series)

        outpng = os.path.join(PLOT_DIR, f"lc_star{ti:04d}.png")
        title  = f"Star {ti:04d} @ (x={xy[ti,0]:.1f}, y={xy[ti,1]:.1f})"
        plot_lightcurve(times, rel, ok, title, outpng, comps_ids=[int(i) for i in comp_ids])
        rel_wide[f"star{ti:04d}"] = rel
        saved += 1

    print(f"Saved {saved} light-curve PNGs to: {PLOT_DIR}")

    if SAVE_WIDE_CSV and saved > 0:
        pd.DataFrame({"JD": times}).to_csv(TIME_CSV_PATH, index=False)
        wide = pd.DataFrame(rel_wide)
        wide.insert(0, "JD", times)
        wide.to_csv(WIDE_CSV_PATH, index=False)
        print(f"Wrote CSVs: {TIME_CSV_PATH} , {WIDE_CSV_PATH}")

if __name__ == "__main__":
    main()


Found 95 light frames.
Detected 197 stars on reference.
Preview saved: ./854b_Basic_out2\detected_stars_preview.png
 processed 10/95 frames...
 processed 20/95 frames...
 processed 30/95 frames...
 processed 40/95 frames...
 processed 50/95 frames...
 processed 60/95 frames...
 processed 70/95 frames...
 processed 80/95 frames...
 processed 90/95 frames...
Kept 197 stars after quality mask.
Saved 189 light-curve PNGs to: ./854b_Basic_out2\plots_allstars_lc
Wrote CSVs: ./854b_Basic_out2\times_jd.csv , ./854b_Basic_out2\allstars_relflux_wide.csv
